### Importing Libraries

In [138]:
import networkx
import rouge
import nltk
import numpy as np
import pandas as pd
from sklearn import preprocessing
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
os.chdir('../Preprocessing')
from normalization import normalize_corpus, parse_content

### Reading the Data

In [139]:
df = pd.read_csv('../Data_collection/dataset.csv')
df.head()

,CATEGORY,CONTENT,SUMMARY
0,business,The Federal Reserve approved Ally Financial In...,The Federal Reserve approved Ally Financial In...
1,business,— Major shareholders of Duke Energy Corp. have...,— Major shareholders of Duke Energy Corp. have...
2,business,Photos taken earlier this month show that Nort...,Photos taken earlier this month show that Nort...
3,business,Thanks to dogged reporting by the Associated P...,Thanks to dogged reporting by the Associated P...
4,business,The energy giant says it is committed to clean...,The energy giant says it is committed to clean...


In [140]:
df['FILTERED_CONTENT'] = df['CONTENT'].apply(parse_content)
df['length'] =df['FILTERED_CONTENT'].apply(len)
df = df[df['length']>15]
df.drop('length',axis =1 ,inplace =True)

In [141]:
#Taking a subset of the data:
df=df.iloc[1:1000]

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 3 to 1992
Data columns (total 4 columns):
CATEGORY            999 non-null object
CONTENT             999 non-null object
SUMMARY             999 non-null object
FILTERED_CONTENT    999 non-null object
dtypes: object(4)
memory usage: 39.0+ KB


In [142]:
df['FILTERED_CONTENT'] = df['FILTERED_CONTENT'].apply(normalize_corpus)
df.head()

,CATEGORY,CONTENT,SUMMARY,FILTERED_CONTENT
3,business,Thanks to dogged reporting by the Associated P...,Thanks to dogged reporting by the Associated P...,[thanks dog report associated press know activ...
6,business,"RALEIGH, N.C., March 26 (Reuters) - Duke Energ...",In a letter to the state’s utilities commissio...,[raleigh march reuters duke energy corp say we...
9,business,"CHARLOTTE, N.C., March 26, 2014 /PRNewswire/ -...","Throughout the past few decades, we have dedic...",[charlotte march duke energy nyse duk today is...
11,business,By Suttinee Yuvejwattana and Michael Sin\n\nMa...,The Japanese satellite detected about a dozen ...,"[by suttinee yuvejwattana michael sin, march b..."
12,business,"PERTH, Australia (AP) � Planes and ships searc...","PERTH, Australia (AP) � Planes and ships searc...",[perth australia ap plane ship search debris s...


### Text Rank Algorithm

In [143]:
def textrank_text_summarizer(data,num_sentences=5):
    
    text =data[3]
    sentences = [sent for sent in nltk.sent_tokenize(data[1])]
    tfidvectorizer = TfidfVectorizer(
                                 ngram_range=(1,5),
                                 smooth_idf=True,
                                 use_idf=True)
    dt_matrix = tfidvectorizer.fit_transform(text)

    similarity_matrix = (dt_matrix * dt_matrix.T)
    similarity_graph = networkx.from_scipy_sparse_matrix(similarity_matrix)
    
    scores = networkx.pagerank(similarity_graph)   
    ranked_sentences = sorted(((score, index) 
                                for index, score 
                                in scores.items()), 
                              reverse=True)
    try:
        top_sentence_indices = [ranked_sentences[index][1] for index in range(num_sentences)]
        top_sentence_indices.sort()
        top_sentences = [sentences[index] for index in top_sentence_indices]
        summary =''.join(top_sentences)
        return summary
    except IndexError:
        pass
    


In [144]:
df['Generated_Summary']= df.apply(textrank_text_summarizer,axis =1)
df.dropna(subset = ['Generated_Summary'],inplace=True)
df.head(10)


,CATEGORY,CONTENT,SUMMARY,FILTERED_CONTENT,Generated_Summary
3,business,Thanks to dogged reporting by the Associated P...,Thanks to dogged reporting by the Associated P...,[thanks dog report associated press know activ...,Thanks to dogged reporting by the Associated P...
6,business,"RALEIGH, N.C., March 26 (Reuters) - Duke Energ...",In a letter to the state’s utilities commissio...,[raleigh march reuters duke energy corp say we...,"RALEIGH, N.C., March 26 (Reuters) - Duke Energ..."
9,business,"CHARLOTTE, N.C., March 26, 2014 /PRNewswire/ -...","Throughout the past few decades, we have dedic...",[charlotte march duke energy nyse duk today is...,We are committed to working with the state of ...
12,business,"PERTH, Australia (AP) � Planes and ships searc...","PERTH, Australia (AP) � Planes and ships searc...",[perth australia ap plane ship search debris s...,"PERTH, Australia (AP) � Planes and ships searc..."
13,business,"PERTH, Australia: Thunderstorms and gale-force...","PERTH, Australia: Thunderstorms and gale-force...",[perth australia thunderstorms wind ground int...,But a major air and sea search has frustrating...
19,business,He said the information had been given to Mala...,RAAF ground crew stand on the apron after an A...,"[he say information give malaysia, raaf ground...",He said the objects were spotted about 200 kil...
20,business,Hints about the lost Malaysian jetliner piled ...,Bad weather cut short the hunt for possible de...,[hint lose malaysian jetliner pile thursday pr...,Hints about the lost Malaysian jetliner piled ...
21,business,Bangkok/Tokyo/Canberra: Over 300 new objects w...,"According to a report from Tokyo, a Japanese s...",[over new object spotted satellite thailand ja...,Bangkok/Tokyo/Canberra: Over 300 new objects w...
25,business,Australian officials coordinating internationa...,Australian officials coordinating internationa...,[australian official coordinate international ...,Australian officials coordinating internationa...
26,business,The Thailand Earth Observation Satellite (Thai...,The Thailand Earth Observation Satellite (Thai...,[the thailand earth observation satellite thai...,The Thailand Earth Observation Satellite (Thai...


In [145]:
print("System Generated Summary:\n")
print(df.iloc[2]['SUMMARY'])
print("\nSummary\n")
print(df.iloc[2]['Generated_Summary'])

System Generated Summary:

Throughout the past few decades, we have dedicated significant resources to the management and monitoring of our ash basins.
We at Duke Energy care deeply about North Carolina and the health and safety of our communities.
Ash management is an energy industry issue that will be impacted by upcoming EPA regulations, some of which are expected in December 2014.
Duke Energy has a long and proud history in North Carolina.
Media contact: Tom WilliamsOffice: 980.373.4743 | 24-Hour: 800.559.3853Analyst contact: Bill CurrensOffice: 704.382.1603Logo - http://photos.prnewswire.com/prnh/20130322/CL81938LOGOSOURCE Duke Energy

Summary

We are committed to working with the state of North Carolina, the North Carolina General Assembly, the North Carolina Utilities Commission and all of our federal and state regulators to fully address the events of the past several weeks as well as reach sound, responsible solutions for how we manage coal ash in our state.We have engaged ind

#### Results

In [146]:
hypothesis = df['Generated_Summary'].tolist()
reference = df['SUMMARY'].tolist()

def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


for aggregator in ['Avg','Best']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

    scores = evaluator.get_scores(hypothesis, reference)
    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        print(prepare_results(results['p'], results['r'], results['f']))
    print()
    


Evaluation with Avg
	rouge-1:	P: 51.62	R: 51.18	F1: 51.14
	rouge-2:	P: 34.62	R: 34.32	F1: 34.30
	rouge-3:	P: 31.05	R: 30.79	F1: 30.77
	rouge-4:	P: 29.43	R: 29.18	F1: 29.16

Evaluation with Best
	rouge-1:	P: 51.62	R: 51.18	F1: 51.14
	rouge-2:	P: 34.62	R: 34.32	F1: 34.30
	rouge-3:	P: 31.05	R: 30.79	F1: 30.77
	rouge-4:	P: 29.43	R: 29.18	F1: 29.16

